In [ ]:
!pip install transformers
!pip install transformers-interpret

In [2]:
import re
import json

In [3]:
from transformers_interpret import SequenceClassificationExplainer
from transformers import AutoModelForSequenceClassification, AutoTokenizer

/home/funicello/.local/share/virtualenvs/ferrara-v9C_kj3N/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
pattern = r".*:.*"

with open("../mlp_the_movie.txt", "rt") as file:
    lines = file.readlines()
    characters = dict()
    data = []
    for line in lines:
        if re.match(pattern,line):
            talker = line.split(":")[0].replace("'","").strip()
            phrase = line.split(":")[-1].strip()
            data.append([talker,phrase])
            if talker in characters:
                characters[talker] = characters[talker]+1
            else:
                characters[talker] = 1

In [5]:
for item in sorted(characters.items(), key=lambda x:x[1]):
    talker = item[0]
    if " and " in talker:
        if "," in talker:
            phrase = talker.replace(" and ",",")
            for p in phrase.split(","):
                p = p.strip()
                if p in characters: characters[p]= characters[p]+1
                else: characters[p] = 1
            del characters[talker]
            continue
        p1, p2 = talker.split(" and ")[0].strip(),talker.split(" and ")[1].strip()
        
        if p1 in characters: characters[p1] = characters[p1]+1
        else: characters[p1] = 1
        
        if p2 in characters: characters[p2] = characters[p2]+1
        else: characters[p2] = 1

        del characters[talker]
    if talker == "Mane Six":
        mane_six = ["Spike","Rainbow Dash","Pinkie Pie","Rarity","Applejack","Twilight Sparkle"]
        for horse in mane_six:
            characters[horse] += item[1]
        del characters[talker]
    if talker == "Mane Six except Twilight":
        mane_six_except = ["Spike","Rainbow Dash","Pinkie Pie","Rarity","Applejack"]
        for horse in mane_six_except:
            characters[horse] += item[1]
        del characters[talker]

In [11]:
with open("character_appearance.json", "w") as outfile:
    json.dump(sorted(characters.items(), key=lambda x:x[1]),outfile)


In [6]:
model_name = "arpanghoshal/EmoRoBERTa"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name,from_tf=True)
cls_explainer = SequenceClassificationExplainer(model,tokenizer)

2023-07-15 15:00:51.273307: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-15 15:00:51.482830: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-15 15:00:52.235261: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
All TF 2.0 model weights were used when initializing RobertaForSequenceClassification.

All the weights of RobertaForSequenceClassification were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, y

In [7]:
for row in data:
    row.append(cls_explainer(row[1]))
    row.append(cls_explainer.id2label[cls_explainer.selected_index])

In [11]:
list_of_json = []
for row in data:
    list_of_json.append(dict(character=row[0],sentence=row[1],attention=row[2],emotion=row[3]))

with open("data.json", "w") as outfile:
    json.dump(list_of_json,outfile)